# Key Information Extraction to SQL Analysis
(Doc Understanding - FinQA)

### Table of contents
1. [Introduction and Prerequisites](#introduction)
2. [Data and Model Preparation](#paragraph1)
    - 2.1 Setup boto3 And Model
    - 2.2 Download Data From Huggingface
    - 2.3 Result Post-processing Preparation
    - 2.4 Ground Turth Data Preparation
3. [Model Execution and Evaluation](#paragraph2)
    - 3.1 Model Execute Preparation
    - 3.2 Model Evaluation Preparation
    - 3.3 Model Execution
4. [Output Analysis Using SQL](#paragraph3)
    - 4.1 Create Table in GLue Data Catalog
    - 4.2 Query on Athena using SQL
5. [(Optional) Model Comparison](#paragraph4)
    - 5.1 Sonnet 3
    - 5.2 Nova lite
    - 5.3 Haiku 3
    - 5.4 Accuracy and Exectuion Time Comparison
    - 5.5 Latency and Token Usage Comparison
6. [Clean Up](#paragraph5)

## 1. Introduction and Prerequisites <a name="introduction"></a>

This notebook demonstrates a proof of concept for processing invoice images utilising GenAI, with the aim of generating structured output that can be seamlessly consumed by downstream SQL engines.

![Architecture](images/architecture.png)

**Prerequisites**
- Bedrock Nova Pro [model access](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html)
- (Optional) Bedrock Claude 3 Sonnet,Claude 3 Haiku and Nova Lite model access.
- A AWS [Glue service role](https://docs.aws.amazon.com/glue/latest/dg/create-an-iam-role.html).
- The Sagemaker exectuion role used in this notebook needs to have [AmazonSageMakerFullAccess](https://docs.aws.amazon.com/aws-managed-policy/latest/reference/AmazonSageMakerFullAccess.html) policy and [turst relationship](https://docs.aws.amazon.com/directoryservice/latest/admin-guide/edit_trust.html) with sagemaker service. Replace `"Service": "ds.amazonaws.com"` to `"Service": "sagemaker.amazonaws.com"` .
- The role above has to have related S3 permissions to create and access the s3 bucket. For example, you can give [AmazonS3FullAccess](https://docs.aws.amazon.com/AmazonS3/latest/userguide/security-iam-awsmanpol.html#security-iam-awsmanpol-amazons3fullaccess).
- The role above has to have Bedrock model access permissions. For example, you can give [AmazonBedrockFullAccess](https://docs.aws.amazon.com/aws-managed-policy/latest/reference/AmazonBedrockFullAccess.html).
- - The role above needs to have permissions to create database, table, cralwer and run crawler permission. For example, you can give [AWSGlueConsoleFullAccess](https://docs.aws.amazon.com/aws-managed-policy/latest/reference/AWSGlueConsoleFullAccess.html)
- The role above needs to have an [iam:PassRole](https://docs.aws.amazon.com/glue/latest/dg/create-an-iam-role.html) permission to use the Glue service role.
~~~
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "ARN OF YOUR GLUE SERVICE ROLE"
        }
    ]
}                                           
~~~

## 2. Data and Model Preparation

### 2.1 Setup boto3 And Model

In [ ]:
import boto3
import json
import time
import pandas as pd

# create model id vars
micro = 'us.amazon.nova-micro-v1:0'
lite = 'us.amazon.nova-lite-v1:0'
pro= 'us.amazon.nova-pro-v1:0'

### 2.2 Download Data From Huggingface

[This dataset](https://huggingface.co/datasets/katanaml-org/invoices-donut-data-v1) contains invoice documents annotated and processed to be ready for Donut ML model fine-tuning. Annotation and data preparation task was done by Katana ML team. For this workshop, we are using a small set of the data for demonstration purposes.

In the dataset, it has two columns. The **image** column already has the image file ocnverted to bytes. The **ground_truth** column holds the actaul contect of the recipt, which will be used to evaluate model results in the later section.

Original dataset info: Kozłowski, Marek; Weichbroth, Paweł (2021), “Samples of electronic invoices”, Mendeley Data, V2, doi: 10.17632/tnj49gpmtz.2

In [ ]:
## TO DO: uncomment this if never run this before
#! pip install huggingface_hub --quiet

In [ ]:
# Download data from huggingface 
splits = {'train': 'data/train-00000-of-00001-a5c51039eab2980a.parquet', 'validation': 'data/validation-00000-of-00001-b8a5c4a6237baf25.parquet', 'test': 'data/test-00000-of-00001-56af6bd5ff7eb34d.parquet'}
# extract test dataset out
dftest = pd.read_parquet("hf://datasets/katanaml-org/invoices-donut-data-v1/" + splits["test"])
#save downloaded files to local
dftest.to_parquet('testdataset.parquet')

In [ ]:
# Have a grasp of the data
print(f"The number of sample invoices: {len(dftest)}")
print("-------------- Data Preview --------------")
dftest.head(5)

### 2.3 Result Post-processing Preparation

In the next section, it will ask the foundation model to return the results in a json format. However, in case extra charachters are returned, the `extract_json_objects` function will only extract the JSON part. Then `json_list_to_dataframe` function will convert the JSON objects into a dataframe, which is going to be fed into downstream SQL engine.

In [ ]:
import re
def extract_json_objects(input_string):
    # Remove the code block markers and any leading/trailing whitespace
    json_string = re.sub(r'```json\n|\n```', '', input_string).strip()
    
    try:
        # Parse the JSON string into a Python object
        json_data = json.loads(json_string)
        
        # Check if the parsed data is a list
        if isinstance(json_data, list):
            return json_data
        else:
            raise ValueError("The extracted JSON is not a list of objects")
    
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

In [ ]:
# convert the json list into a dataframe
def json_list_to_dataframe(json_list):
    """
    Convert a list of JSON objects to a pandas DataFrame.

    Args:
        json_list (list): A list of JSON objects (dictionaries).

    Returns:
        pandas.DataFrame: A pandas DataFrame constructed from the list of JSON objects.
    """
    # Create an empty DataFrame
    df = pd.DataFrame()

    # If the input list is empty, return the empty DataFrame
    if not json_list:
        return df

    # Extract the keys from the first JSON object
    keys = list(json_list[0].keys())

    # Create a DataFrame from the list of JSON objects
    df = pd.DataFrame(json_list, columns=keys)

    return df

### 2.4 Ground Turth Data Preparation

The `ground_truth` function below extracts the ground truth column, which will later be used to compare with the model output to evaluate the model's accuracy.

In [ ]:
# prepare the Ground Truth data into a dataframe
def ground_truth(gt_data):
    # Parse the JSON string
    data = json.loads(gt_data)

    # Extract the necessary information
    header = data.get('gt_parse', {}).get('header', {})
    items = data.get('gt_parse', {}).get('items', [])

    # Create a list of dictionaries for the DataFrame
    records = []
    for item in items:
        record = {
            'invoice_number': header.get('invoice_no', ''),
            'date_of_issue': header.get('invoice_date', ''),
            'seller_name': header.get('seller', ''),
            'client_name': header.get('client', ''),
            'item_description': item.get('item_desc', ''),
            'quantity': get_float_value(item.get('item_qty', '0'), remove_spaces=True),
            'net_price': get_float_value(item.get('item_net_price', '0'), remove_spaces=True),
            'vat_percent': item.get('item_vat', item.get('iban',0)),
            'gross_worth': get_float_value(item.get('item_gross_worth', '0'), remove_spaces=True),
        }
        records.append(record)
    
    # Create the DataFrame
    gt_df = pd.DataFrame(records)
    return gt_df

def get_float_value(value, remove_spaces=False):
    if remove_spaces:
        value = value.replace(' ', '')
    try:
        return float(value.replace(',', '.'))
    except ValueError:
        return 0.0

## 3. Model Execution and Evaluation <a name="paragraph1"></a>
This section define the functions that run invoice images OCR use `model_id` specified. The model results is collected which is later used to comparewith the ground truth data, and evaluate the model's performance by calcuating the accuracy rate.

### 3.1 Model Execute Preparation

In [ ]:
# define the model
def run_model(model_id, system_list,prompt_text, image_df, inf_params, client):
    
    final_result_df_list = []
    final_gt_df_list = []
    final_mdoel_eval_df_list = []

    start = time.time()
    

    for index, row in image_df.iterrows():
        
        imagedata = row["image"]['bytes']
        print(f"Processing invoice {index}")
        
        #execute model
        message_list = [{
            "role": "user",
            "content": [
                {
                    "image": {
                        "format": "jpeg",
                        "source": {"bytes": imagedata},
                    }
                },
                {
                    "text": prompt_text
                }
            ],
        }]


        response = client.converse(modelId=model_id, 
                                   messages=message_list, 
                                   system = system_list, 
                                   inferenceConfig = inf_params)

        content_text = response["output"]["message"]["content"][0]["text"]
        # print(content_text)
        
         # post processing the result
        json_lst = extract_json_objects(content_text)
        result_df = json_list_to_dataframe(json_lst)
        
        #add metrics
        result_df['input_token_usage'] = int(response['usage']['inputTokens'])
        result_df['output_token_usage'] = int(response['usage']['outputTokens'])
        result_df['model_latency'] = int(response['metrics']['latencyMs'])
        
        final_result_df_list.append(result_df)
        
        # get the ground truth data
        gtdata = row["ground_truth"]
        # print(gtdata)
        gt_df = ground_truth(gtdata)
        final_gt_df_list.append(gt_df)
        
        # compare the model results with gt results
        mdoel_eval_df = model_evaluation(result_df, gt_df)
        final_mdoel_eval_df_list.append(mdoel_eval_df)
    
    end = time.time()
    execution_time = end - start
    print("Model: ",model_id, "took", execution_time, "seconds to finish processing", len(image_df), "invoice pictures")
    
    final_result_df = pd.concat(final_result_df_list, axis=0, ignore_index=True)   
    final_gt_df = pd.concat(final_gt_df_list, axis=0, ignore_index=True)
    final_mdoel_eval_df = pd.concat(final_mdoel_eval_df_list, axis=0, ignore_index=True)
    
    return final_result_df,final_gt_df,final_mdoel_eval_df,execution_time

### 3.2 Model Evaluation Preparation

In [ ]:
# combine the model results with GroundTruth for comparation
def model_evaluation(result_df, gt_df):
    # Reset the index of both dataframes
    result_df = result_df.reset_index()
    gt_df = gt_df.reset_index()
    
    # Merge the dataframes on 'Invoice Number' and index
    try:
        merged_df = result_df.merge(gt_df, on=['index'], how='left', suffixes=('_result', '_gt'))
    except:
        result_df['invoice_number'] = result_df['invoice_number'].astype(str)
        result_df['index'] = result_df['index'].astype(str)
        gt_df['invoice_number'] = gt_df['invoice_number'].astype(str)
        gt_df['index'] = gt_df['index'].astype(str)
        merged_df = result_df.merge(gt_df, on=[ 'index'], how='left', suffixes=('_result', '_gt'))
    # Rename the columns to distinguish them
    renamed_cols = {f'{col}_result': f'{col}_result' if col != 'index' else col for col in merged_df.columns}
    renamed_cols.update({f'{col}_gt': f'{col}_gt' if col != 'index' else col for col in merged_df.columns})
    merged_df = merged_df.rename(columns=renamed_cols)
    
    # Rearrange the columns to make them more comparable
    cols_to_keep = [
        'invoice_number_result', 
        'invoice_number_gt',
        'date_of_issue_result', 
        'date_of_issue_gt', 
        'seller_name_result', 
        'seller_name_gt', 
        'client_name_result', 
        'client_name_gt', 
        'item_description_result', 
        'item_description_gt', 
        'quantity_result', 
        'quantity_gt', 
        'net_price_result', 
        'net_price_gt', 
        'vat_percent_result', 
        'vat_percent_gt', 
        'gross_worth_result', 
        'gross_worth_gt']
    merged_df = merged_df[cols_to_keep]
    
    return merged_df

In [ ]:
def model_score(final_mdoel_eval_df):
    # add column names that you want to evaluate
    eval_list = ["invoice_number", "quantity", "net_price", "vat_percent", "gross_worth"]
    # compare if model result is consistent with ground truth
    for col in eval_list:
        if col in ['vat_percent', 'invoice_number'] :
            final_mdoel_eval_df[col+ "_diff"] = final_mdoel_eval_df[col + "_result"] == final_mdoel_eval_df[col + "_gt"]
        else: 
            difftest = final_mdoel_eval_df[col + "_result"].astype(float) - final_mdoel_eval_df[col + "_gt"].astype(float)
            final_mdoel_eval_df[col + "_diff"] = (difftest == 0)
            
    # return the rows that the model results and ground truth are not consistent
    df_wrong = final_mdoel_eval_df[final_mdoel_eval_df.eq(False).any(axis=1)]
    accuracy = (1 - len(df_wrong)/len(final_mdoel_eval_df)) * 100
    print(f"Accuracy is {accuracy}%")
    return accuracy, df_wrong

### 3.3 Model Execution
This section will utilise all the functions we define above to invoke model. The accuracy is calculated and the OCR result is stored as `output.parquet` file in s3. 

In [ ]:
from botocore.config import Config
config = Config(retries = {'max_attempts': 10,'mode': 'adaptive'})
client = boto3.client("bedrock-runtime",region_name="us-east-1",config=config)

In [ ]:
image_df = dftest.copy()
# to run on a small scale
# image_df = dftest.iloc[1:2]

In [ ]:
# Setting up the parameters
system_list = [
    {
        "text": "You are an data expert who is good at analysing the data and design relational databases"
    }
]

prompt_text = '''
please go through the full picture and provide ALL the contect and items you see. Missing item will be penalised.

The image is a invoice document and please extract below informiaton in the form of JSON. 
Each item should return one JSON object, and if there are multiple items in the invoice, retrun a list of JSON objects.

The result has to follow below example structure:
**Example 1:**
[{"invoice_number": "65321852",
	"date_of_issue": "04/11/2021",
	"seller_name": "Kaufman Cooper and Young",
	"client_name": "Wells Carlson",
	"item_description": "New KID CUDI Size S to 3XL",
	"quantity": 2.00,
	"net_price": 22.49,
    "vat_percent": "10%",
    "gross_worth": 49.48
    },
    {"invoice_number": "65321852",
	"date_of_issue": "04/11/2021",
	"seller_name": "Kaufman Cooper and Young",
	"client_name": "Wells Carlson",
	"item_description": "New KID CUDI Size S to 3XL",
	"quantity": 2.00,
	"net_price": 22.49,
    "vat_percent": "10%",
    "gross_worth": 49.48
    }
    ]

Now, strictly using the examples format above, and retrun a list of JSON objects. Please keep value decimal as it, do not round up or down anything.
All property names must be enclosed in double quotes.
If threre's any special charachters in the item_description, such as "/", "\", and ",", please remove them.
The JSON string should be valid and properly formatted, ready to be parsed by JSON.parse() in JavaScript or json.loads() in Python.
Check the JSON string are proper JSON format; if not, please fix it.
The "vat_percent" key stands for value-added tax, which is always in percentage (0% - 100%). If the recipt only shows a number, add "%" to the value.
For "seller_name" and "client_name" do not include address, just output the company name.
Do not give filed that not listed above. Do not output any explaination or other text other than the json.

'''

inf_params = {"maxTokens": 3000, "topP": 0.1, "temperature": 0.01}

In [ ]:
# choose model
model_id = pro

'''
Execute model to extract info and construct formated csv output
The output contains 3 tables, 

1. model final_result_df, 
2. groundtruth final_gt_df, 
3. concate table of model final_result_df and groundtruth

and timespent
'''

nova_final_result_df,\
nova_final_gt_df,\
nova_final_mdoel_eval_df,\
nova_exe_time = run_model(model_id, 
                         system_list,prompt_text, 
                         image_df, 
                         inf_params, 
                         client)

accuracy, df_wrong = model_score(nova_final_mdoel_eval_df)

Nova = {
"model_id":model_id,
"accuracy(%)":accuracy,
"execution_time(s)": nova_exe_time
}

### 4.1 Create Table in Glue Data Catalog

In this section, the model result from the previous section will be saved into a S3 location. For this, a S3 bucket will be created. Then a Glue database and a crawler will be created. The Glue cralwer will crawl the data from the S3 location, and create a table on Glue and Athena.

In [ ]:
import uuid
myuuid = uuid.uuid4()

# create S3 bucket
s3_client = boto3.client("s3")
bucket_name = f"bedrock-invoice-{myuuid}"
prefix_name = "invoice"
s3_client.create_bucket(Bucket=bucket_name)
print(f"AWS S3 bucket '{bucket_name}' created succesfully.")

# Create the database
glue_client = boto3.client("glue")
database_name = "bedrock_invoice_db"
glue_client.create_database(DatabaseInput={'Name': database_name})
print(f"AWS Glue database '{database_name}' created succesfully.")

In [ ]:
# Save the result to the S3 bucket
s3_target_path = f"s3://{bucket_name}/{prefix_name}/"
final_output = nova_final_result_df.drop(['input_token_usage', 'output_token_usage','model_latency'], axis = 1)
final_output.to_parquet(f"{s3_target_path}output.parquet")

In [ ]:
# Create the crawler
# Add the Glue service role here created in prerequisites 
glue_svc_role = "YOUR_GLUE_SERVICE_ROLE_ARN"
crawler_name = 'bedrock-invoice-crawler'
response = glue_client.create_crawler(
    Name=crawler_name,
    Role=glue_svc_role,
    DatabaseName=database_name,
    Description='Crawler for S3 data',
    Targets={
        'S3Targets': [
            {'Path': s3_target_path}
        ]
    }
)
time.sleep(5)
print(f"AWS Glue Crawler '{crawler_name}' created succesfully.")

In [ ]:
# Run the Glue Crawler. This step is going to take a few minutes to complete. 
response = glue_client.start_crawler(
    Name=crawler_name
)
print(f"AWS Glue Crawler '{crawler_name}' started.")
time.sleep(5)
state_previous = None
while True:
    response_get = glue_client.get_crawler(Name=crawler_name)
    state = response_get["Crawler"]["State"]
    if state != state_previous:
        state_previous = state
    if state == "READY":  # Other known states: RUNNING, STOPPING
        break
    time.sleep(10)

print(f"AWS Glue Crawler '{crawler_name}' finished.")

### 4.2 Query on Athena using SQL

Go to Athena console, and check the table has been created.

![Athena](images/athena_1.png)

**Analysis one: find the total sales for seller Jackson Ltd**

~~~
SELECT ROUND(SUM(gross_worth), 2) AS total_sales FROM invoice 
WHERE seller_name = 'Jackson Ltd'
~~~

![Athena](images/athena_2.png)

**Analysis two: find all the items sold by 'Jackson Ltd'**

~~~
SELECT DISTINCT item_description FROM invoice 
WHERE seller_name = 'Jackson Ltd'
~~~

![Athena](images/athena_3.png)

## 5.(Optional) Model Comparison 
Below section will compare the performance of the Claude Sonnet 3 model, Nova Pro model, Nova Lite model and Claude Haiku 3 model on processing results using the same dataset. The metrics we will focus on are `accuracy` and model `exectuion time`, `output_token_usage` (from a cost perspective) and `model_latency` (from a performance perspective) on each invoice level.

### 5.1 Sonnet 3

In [ ]:
# choose model
model_id = 'us.anthropic.claude-3-sonnet-20240229-v1:0'

# run model
sonnet3_final_result_df,\
sonnet3_final_gt_df,\
sonnet3_final_mdoel_eval_df,\
sonnet3_exe_time = run_model(model_id, 
                           system_list,
                           prompt_text, 
                           image_df, 
                           inf_params, 
                           client)

accuracy, df_wrong = model_score(sonnet3_final_mdoel_eval_df)
sonnet3 = {
"model_id":model_id,
"accuracy(%)":accuracy,
"execution_time(s)": sonnet3_exe_time
}

In [ ]:
# print out the records that model results are inconsistent with ground truth
# df_wrong

### 5.2 Nova lite

In [ ]:
# choose model
model_id = lite

# run model
novalite_final_result_df,\
novalite_final_gt_df, \
novalite_final_mdoel_eval_df,\
novalite_exe_time = run_model(model_id, 
                               system_list,
                               prompt_text, 
                               image_df, 
                               inf_params, 
                               client)

accuracy, df_wrong = model_score(novalite_final_mdoel_eval_df)
novalite = {
"model_id":model_id,
"accuracy(%)":accuracy,
"execution_time(s)": novalite_exe_time
}

### 5.3 Haiku 3

In [ ]:
# choose model
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

# run model
haiku3_final_result_df,\
haiku3_final_gt_df, \
haiku3_final_mdoel_eval_df,\
haiku3_exe_time = run_model(model_id, 
                               system_list,
                               prompt_text, 
                               image_df, 
                               inf_params, 
                               client)

accuracy, df_wrong = model_score(haiku3_final_mdoel_eval_df)
haiku3 = {
"model_id":model_id,
"accuracy(%)":accuracy,
"execution_time(s)": haiku3_exe_time
}

### 5.4 Accuracy and Exectuion Time Comparison

In [ ]:
# Add or remove model metrics from the list below to show the final metrics comparison
comparison_list = [sonnet3, Nova, novalite, haiku3]
model_comp = json_list_to_dataframe(comparison_list)
model_comp

### 5.5 Latency and Token Usage Comparison

In [ ]:
metrics_df = pd.DataFrame({'invoice_number': nova_final_result_df['invoice_number'], 
                           
                           'sonnet3_output_token_usage': sonnet3_final_result_df['output_token_usage'], 
                           'novapro_output_token_usage': nova_final_result_df['output_token_usage'], 
                           'novalite_output_token_usage': novalite_final_result_df['output_token_usage'], 
                           'haiku3_output_token_usage': haiku3_final_result_df['output_token_usage'], 
                           
                           'sonnet3_input_token_usage': sonnet3_final_result_df['input_token_usage'], 
                           'novapro_input_token_usage': nova_final_result_df['input_token_usage'], 
                           'novalite_input_token_usage': novalite_final_result_df['input_token_usage'],
                           'haiku3_input_token_usage': haiku3_final_result_df['input_token_usage'],
                           
                           'sonnet3_model_latency': sonnet3_final_result_df['model_latency'],
                           'novapro_model_latency': nova_final_result_df['model_latency'],
                           'novalite_model_latency': novalite_final_result_df['model_latency'],
                           'haiku3_model_latency': haiku3_final_result_df['model_latency'],
                           
                          })

#group table value by invoice number 
grouped = metrics_df.groupby('invoice_number').mean().reset_index()

In [ ]:
import matplotlib.pyplot as plt

# change 'invoice_number' to string
metrics_df['invoice_number'] = metrics_df['invoice_number'].astype(str)

# create two fig
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

# plot Token Usage comparison fig
ax1.plot(metrics_df['invoice_number'], metrics_df['sonnet3_output_token_usage'], marker='o', color='red', label='sonnet3')
ax1.plot(metrics_df['invoice_number'], metrics_df['novapro_output_token_usage'], marker='o', color='yellow', label='Novapro')
ax1.plot(metrics_df['invoice_number'], metrics_df['novalite_output_token_usage'], marker='o', color='green', label='Novalite')
ax1.plot(metrics_df['invoice_number'], metrics_df['haiku3_output_token_usage'], marker='o', color='blue', label='haiku3')

ax1.axhline(y=metrics_df['sonnet3_output_token_usage'].mean(), color='lightcoral', linestyle='--', label='sonnet3 Mean') 
ax1.axhline(y=metrics_df['novapro_output_token_usage'].mean(), color='yellow', linestyle='--', label='Nova pro Mean') 
ax1.axhline(y=metrics_df['novalite_output_token_usage'].mean(), color='lightgreen', linestyle='--', label='Nova lite Mean') 
ax1.axhline(y=metrics_df['haiku3_output_token_usage'].mean(), color='blue', linestyle='--', label='haiku3 Mean') 


ax1.set_xlabel('Invoice Number')
ax1.set_ylabel('Output Token Usage')
ax1.set_title('Output Token Usage Comparison')
ax1.legend()
ax1.grid(True)
ax1.tick_params(axis='x', labelrotation=90)

# plot Model Latency comparison fig
ax2.plot(metrics_df['invoice_number'], metrics_df['sonnet3_model_latency'], marker='o', color='red', label='sonnet3')
ax2.plot(metrics_df['invoice_number'], metrics_df['novapro_model_latency'], marker='o', color='yellow', label='Novapro')
ax2.plot(metrics_df['invoice_number'], metrics_df['novalite_model_latency'], marker='o', color='green', label='Novalite')
ax2.plot(metrics_df['invoice_number'], metrics_df['haiku3_model_latency'], marker='o', color='blue', label='haiku3')


ax2.axhline(y=metrics_df['sonnet3_model_latency'].mean(), color='lightcoral', linestyle='--', label='sonnet3 Mean') 
ax2.axhline(y=metrics_df['novapro_model_latency'].mean(), color='yellow', linestyle='--', label='Nova pro Mean')
ax2.axhline(y=metrics_df['novalite_model_latency'].mean(), color='lightgreen', linestyle='--', label='Nova lite Mean') 
ax2.axhline(y=metrics_df['haiku3_model_latency'].mean(), color='blue', linestyle='--', label='haiku3 Mean') 


ax2.set_xlabel('Invoice Number')
ax2.set_ylabel('Model Latency (ms)')
ax2.set_title('Model Latency Comparison')
ax2.legend()
ax2.grid(True)
ax2.tick_params(axis='x', labelrotation=90)

# show plot
plt.tight_layout()
plt.show()

# 6. Clean Up

### Delete s3 bucket

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)

# Delete all objects in the bucket
bucket.objects.all().delete()

# Delete the bucket
response = s3_client.delete_bucket(Bucket=bucket_name)

### Delete Glue Crawler

In [ ]:
response = glue_client.delete_crawler(Name=crawler_name)
print(f"Deleting crawler: {crawler_name}")

### Delete AWS Glue Data Catalog tables and database

In [ ]:
def delete_tables_in_database(database_name):
    # Get a list of all tables in the database
    tables = glue_client.get_tables(DatabaseName=database_name)['TableList']

    # Iterate over the tables and delete each one
    for table in tables:
        table_name = table['Name']
        glue_client.delete_table(DatabaseName=database_name, Name=table_name)
        print(f"Deleting table: {table_name}")


delete_tables_in_database(database_name)
response = glue_client.delete_database(Name=database_name)
print(f"Deleting database: {database_name}")